In [25]:
pip install -q datasets transformers peft accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.9 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dep

In [ ]:
import json
from pathlib import Path

def convert_finqa(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)

    converted = []
    for item in raw_data:
        # Soru
        question = item["qa"]["question"]
        answer = item["qa"].get("answer", "")

        # Tabloyu string olarak birleştir
        table = item.get("table", [])
        table_text = "\n".join([" | ".join(row) for row in table])

        # Kaydet
        converted.append({
            "context": table_text,
            "question": question,
            "answer": answer
        })

    # JSONL olarak kaydet
    with open(output_path, 'w', encoding='utf-8') as f:
        for row in converted:
            f.write(json.dumps(row, ensure_ascii=False) + '\n')

# Yollar
base_path = "/kaggle/input/finqa-ds"
out_path = "/kaggle/working"

# Dönüştürme işlemleri
convert_finqa(f"{base_path}/train.json", f"{out_path}/train_converted.jsonl")
convert_finqa(f"{base_path}/dev.json", f"{out_path}/dev_converted.jsonl")
convert_finqa(f"{base_path}/test.json", f"{out_path}/test_converted.jsonl")

print("✔️ Dönüştürme tamamlandı.")

In [26]:
from datasets import load_dataset

# JSONL dosyalarını HuggingFace formatında yükle
data_files = {
    "train": "/kaggle/working/train_converted.jsonl",
    "validation": "/kaggle/working/dev_converted.jsonl",
    "test": "/kaggle/working/test_converted.jsonl"
}

raw_datasets = load_dataset("json", data_files=data_files, split={"train": "train", "validation": "validation", "test": "test"})

# 🧠 Input-output formatına dönüştür
def preprocess(example):
    return {
        "input": f"Table:\n{example['context']}\n\nQuestion: {example['question']}",
        "output": example["answer"]
    }

tokenized_datasets = raw_datasets.map(preprocess)

# Kontrol: Örnek göster
print(tokenized_datasets["train"][0])


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/6251 [00:00<?, ? examples/s]

Map:   0%|          | 0/883 [00:00<?, ? examples/s]

Map:   0%|          | 0/1147 [00:00<?, ? examples/s]

{'context': ' | october 31 2009 | november 1 2008\nfair value of forward exchange contracts asset ( liability ) | $ 6427 | $ -23158 ( 23158 )\nfair value of forward exchange contracts after a 10% ( 10 % ) unfavorable movement in foreign currency exchange rates asset ( liability ) | $ 20132 | $ -9457 ( 9457 )\nfair value of forward exchange contracts after a 10% ( 10 % ) favorable movement in foreign currency exchange rates liability | $ -6781 ( 6781 ) | $ -38294 ( 38294 )', 'question': 'what is the the interest expense in 2009?', 'answer': '380', 'input': 'Table:\n | october 31 2009 | november 1 2008\nfair value of forward exchange contracts asset ( liability ) | $ 6427 | $ -23158 ( 23158 )\nfair value of forward exchange contracts after a 10% ( 10 % ) unfavorable movement in foreign currency exchange rates asset ( liability ) | $ 20132 | $ -9457 ( 9457 )\nfair value of forward exchange contracts after a 10% ( 10 % ) favorable movement in foreign currency exchange rates liability | $ -

In [27]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType

model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none"
)

model = get_peft_model(base_model, peft_config)

model.print_trainable_parameters()

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

trainable params: 4,718,592 || all params: 787,868,672 || trainable%: 0.5989


In [37]:
def tokenize(sample):
    model_input = tokenizer(
        sample["input"],
        max_length=256,        
        padding="max_length",
        truncation=True,
    )
    labels = tokenizer(
        sample["output"],
        max_length=64,         
        padding="max_length",
        truncation=True
    )
    model_input["labels"] = labels["input_ids"]
    return model_input


# Tokenize işlemi
tokenized_dataset = tokenized_datasets.map(tokenize, batched=True, remove_columns=tokenized_datasets["train"].column_names)

Map:   0%|          | 0/6251 [00:00<?, ? examples/s]

In [29]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
import torch

# GPU varsa otomatik kullan
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Veri toplayıcı (otomatik padding ve label shifting için)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [40]:
training_args = TrainingArguments(
    output_dir="/kaggle/working/finbot-lora-checkpoints",
    per_device_train_batch_size=1,        
    per_device_eval_batch_size=1,           
    gradient_accumulation_steps=8,         
    num_train_epochs=3,
    learning_rate=5e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    save_total_limit=2,
    fp16=False,                         
    report_to="none",
    push_to_hub=False
)


In [41]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

/tmp/ipykernel_36/1815454466.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [42]:
import torch
torch.cuda.empty_cache()

In [43]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,13.962200,5.850568
2,2.320000,0.296526
3,0.294000,0.252665


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=1173, training_loss=7.753166102793922, metrics={'train_runtime': 10346.4196, 'train_samples_per_second': 1.813, 'train_steps_per_second': 0.113, 'total_flos': 2.174657388399821e+16, 'train_loss': 7.753166102793922, 'epoch': 3.0})

In [44]:
save_path = "/kaggle/working/finbot-peft-model"

model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"Model ve tokenizer '{save_path}' klasörüne kaydedildi.")

Model ve tokenizer '/kaggle/working/finbot-peft-model' klasörüne kaydedildi.
